In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/Loss Function Evaluation/Data/Correlation and statistical test/
!ls

In [ ]:
!pip install pingouin
!pip install dcor
!pip install vaex
!pip install xarray
!pip install pyspark
!pip install geopandas
!pip install causalimpact
!pip install tensorflow
!pip install tensorflow-probability
!pip install torch torchvision torchaudio
!pip install keras
!pip install dask
!pip install seaborn


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import json
import scipy.stats as sc
import statsmodels.api as sm
import pingouin as pg
import dcor
import vaex
import xarray as xr
import geopandas as gpd
import tensorflow as tf
import tensorflow_probability as tfp
import torch
import dask.array as da
import seaborn as sns
from scipy.stats import rankdata
from sklearn.metrics import pairwise_distances
from scipy.stats import entropy
from sklearn.metrics import mutual_info_score
from scipy.spatial.distance import pdist, squareform
from sklearn.preprocessing import StandardScaler
from scipy.stats import rankdata
from keras import backend as K
from causalimpact import CausalImpact
from pyspark.sql import SparkSession
from pyspark.sql.functions import corr
from scipy.stats import pearsonr, spearmanr, kendalltau, pointbiserialr
from sklearn.datasets import make_regression
from sklearn.feature_selection import mutual_info_regression

In [ ]:
# Data 1

mri_features = pd.read_excel('/content/drive/MyDrive/Loss Function Evaluation/Data/Classification/Multiclass/MRI_Radiomics Features.xlsx')
us_mri_features = pd.read_excel('/content/drive/MyDrive/Loss Function Evaluation/Data/Classification/Multiclass/US_MRI_Radiomics Features.xlsx')
ucla_data = pd.read_excel('/content/drive/MyDrive/Loss Function Evaluation/Data/Classification/Multiclass/UCLA.xlsx')

print(f'MRI_Features: {mri_features.shape}')
print(f'us_mri_features: {us_mri_features.shape}')
print(f'ucla_data: {ucla_data.shape}')

# Update lesion names in ucla_data
ucla_data['lesion_name'] = ucla_data['lesion_name'].str.replace('_US_Prostate', '')

# Merge data based on lesion_name and PatientID
merged_data = ucla_data.merge(mri_features, left_on='lesion_name', right_on='PatientID')


# Remove a specific class (for example, class 1) from multi-class data
class_to_remove = 1
filtered_data = merged_data[merged_data['UCLA Score (Similar to PIRADS v2)_Multi-class'] != 0]
filtered_data = filtered_data[filtered_data['UCLA Score (Similar to PIRADS v2)_Multi-class'] != 1]
filtered_data = filtered_data[filtered_data['UCLA Score (Similar to PIRADS v2)_Multi-class'] != 2]


# Drop name columns and separate features and target variables
features_filtered = filtered_data.drop(columns=['lesion_name', 'PatientID', 'Labels_Binary', 'Label'])
# y_true_multi_filtered = filtered_data['UCLA Score (Similar to PIRADS v2)_Multi-class']
y_true_bi_filtered = filtered_data['Labels_Binary']

# Split data into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(features_filtered, y_true_multi_filtered, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(features_filtered, y_true_bi_filtered, test_size=0.2, random_state=42)
y_true_1 = y_test
y_true_1 = np.array(y_true_1)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
np.random.seed(42)
x1 = np.random.rand(200)
y1 = np.random.rand(200)
df = pd.DataFrame({'x': x1, 'y': y1})

x2 = features_filtered['morph_area_mesh']
y2 = features_filtered['ngl_dcnu_3D']
x2 = x2.to_numpy()
y2 = y2.to_numpy()

x3 = features_filtered['ngl_hdhge_3D']
y3 = features_filtered['ngl_glnu_3D']
x3 = x3.to_numpy()
y3 = y3.to_numpy()

x4 = features_filtered['morph_av']
y4 = features_filtered['ngl_lde_3D']
x4 = x4.to_numpy()
y4 = y4.to_numpy()



df1 = pd.DataFrame({'x': x1, 'y': y1})
df2 = pd.DataFrame({'x': x2, 'y': y2})
df3 = pd.DataFrame({'x': x3, 'y': y3})
df4 = pd.DataFrame({'x': x4, 'y': y4})


with pd.ExcelWriter('/content/drive/MyDrive/Loss Function Evaluation/Data/Correlation and statistical test/Corr_output.xlsx') as writer:
    df1.to_excel(writer, sheet_name='random', index=False)
    df2.to_excel(writer, sheet_name='area_mesh-ngl_dcnu', index=False)
    df3.to_excel(writer, sheet_name='ngl_hdhge-ngl_glnu', index=False)
    df4.to_excel(writer, sheet_name='morph_av-ngl_lde', index=False)

In [ ]:
data_corr = pd.read_excel('/content/drive/MyDrive/Loss Function Evaluation/Data/Correlation and statistical test/Corr_output.xlsx', sheet_name="morph_av-ngl_lde")
x2 = data_corr['x']
y2 = data_corr['y']
x2 = np.array(x2)
y2 = np.array(y2)

In [ ]:
x = x2
y = y2
np.random.seed(42)
binary_array = np.random.randint(2, size = x.shape)

Scipy Library:

In [ ]:
def scipy_corrs (x, y, binary_array):
  return {
      'Pearson': sc.pearsonr (x, y)[0],
      'Spearman': sc.spearmanr (x, y)[0],
      'Kendall’s Tau': sc.kendalltau (x, y)[0],
      'Point-Biserial': sc.pointbiserialr (binary_array, y)[0]
  }


Scikit-Learn Library:

In [ ]:
def spearman_correlation_sklearn(x, y):
    # Convert to numpy arrays
    x = np.array(x)
    y = np.array(y)

    # Rank the data
    x_rank = rankdata(x)
    y_rank = rankdata(y)

    # Compute Pearson correlation coefficient on the ranks
    spearman_corr = np.corrcoef(x_rank, y_rank)[0, 1]

    return spearman_corr

def pearson_correlation_sklearn(x, y):
    """
    Compute Pearson's correlation coefficient using sklearn for preprocessing.

    Parameters:
    x (list or np.array): Continuous variable x.
    y (list or np.array): Continuous variable y.

    Returns:
    float: Pearson's correlation coefficient.
    """
    # Convert to numpy arrays if they are not already
    x = np.array(x)
    y = np.array(y)

    # Standardize the data
    scaler = StandardScaler()
    x_scaled = scaler.fit_transform(x.reshape(-1, 1)).flatten()
    y_scaled = scaler.fit_transform(y.reshape(-1, 1)).flatten()

    # Compute Pearson correlation coefficient
    pearson_corr = np.corrcoef(x_scaled, y_scaled)[0, 1]

    return pearson_corr



def sklearn_corrs (x, y):
  return {
      'Pearson': pearson_correlation_sklearn(x, y),
      'Spearman': spearman_correlation_sklearn(x, y) ,
      'Mutual Information': mutual_info_score(x, y)
  }


Pandas Library:

In [ ]:
def distance_correlation(X, Y):
    X = np.asarray(X)
    Y = np.asarray(Y)

    def distance_matrix(V):
        return np.abs(V[:, None] - V[None, :])

    def centered_distance_matrix(D):
        n = D.shape[0]
        mean_i = D.mean(axis=1)
        mean_j = D.mean(axis=0)
        mean_ij = D.mean()
        return D - mean_i[:, None] - mean_j[None, :] + mean_ij

    A = distance_matrix(X)
    B = distance_matrix(Y)

    A_centered = centered_distance_matrix(A)
    B_centered = centered_distance_matrix(B)

    n = X.shape[0]
    dCov_AB = np.sum(A_centered * B_centered) / (n ** 2)

    dVar_A = np.sum(A_centered ** 2) / (n ** 2)
    dVar_B = np.sum(B_centered ** 2) / (n ** 2)

    if dVar_A > 0 and dVar_B > 0:
        dCor = np.sqrt(dCov_AB) / np.sqrt(np.sqrt(dVar_A) * np.sqrt(dVar_B))
    else:
        dCor = 0.0

    return dCor




def pandas_corrs (x, y, binary_array):
    df = pd.DataFrame({'x': x, 'y': y})

    pxy =  pd.crosstab(x, y)/  pd.crosstab(x, y).sum().sum()
    px = pxy.sum(axis = 1)
    py = pxy.sum(axis = 0)

    mi = 0
    for i in px.index:
        for j in py.index:
            pxy_ij = pxy.loc[i, j] if (i in pxy.index and j in pxy.columns) else 0
            if pxy_ij > 0:
                mi += pxy_ij * np.log(pxy_ij / (px[i] * py[j]))

    return {
      'Pearson': df['x'].corr(df['y'], method ='pearson'),
      'Spearman': df['x'].corr(df['y'], method ='spearman'),
      'Kendall’s Tau': df['x'].corr(df['y'], method ='kendall'),
      'Point-Biserial': (y[binary_array == 1].mean()- y[binary_array == 0].mean()) / y.std() * np.sqrt(binary_array.sum()   * (len(binary_array) - binary_array.sum()) / len(binary_array)**2),
      'Mutual Information': mi,
      'Distance Correlation': distance_correlation(x, y)
    }


Numpy Library:

In [ ]:
def numpy_corrs (x, y) :
    x_rank = rankdata(x)
    y_rank = rankdata(y)
    tau, p = kendalltau(x_rank, y_rank)
    # calculating MI
    def mutual_information(x, y):
        # Ensure that x and y are numpy arrays
        x = np.asarray(x)
        y = np.asarray(y)

        # Find unique values and their frequencies (probabilities)
        p_x, x_counts = np.unique(x, return_counts=True)
        p_x = x_counts / len(x)  # Marginal probability distribution for x

        p_y, y_counts = np.unique(y, return_counts=True)
        p_y = y_counts / len(y)  # Marginal probability distribution for y

        # Joint probability distribution
        joint_xy, _, _ = np.histogram2d(x, y, bins=[len(p_x), len(p_y)])
        joint_xy /= len(x)  # Normalize to get joint probabilities

        # Calculate mutual information
        mi = 0.0
        for i in range(len(p_x)):
            for j in range(len(p_y)):
                if joint_xy[i, j] > 0:  # Avoid log(0)
                    mi += joint_xy[i, j] * np.log(joint_xy[i, j] / (p_x[i] * p_y[j]))

        return mi

    return {
        'Pearson': np.corrcoef(x, y)[0, 1],
        #custom implementation
        'Spearman': np.corrcoef(x_rank, y_rank)[0, 1],
        'Kendall’s Tau': tau,
        'Mutual Information':mutual_information(x, y),
        #'Distance Correlation': distance_correlation(x, y)
    }

Statsmodels Library:

In [ ]:
 def kendall_tau_st(X, Y):
    X_rank = pd.Series(X).rank()
    Y_rank = pd.Series(Y).rank()
    n = len(X)
    concordant = 0
    discordant = 0

    for i in range(n):
        for j in range(i + 1, n):
            if (X_rank[i] < X_rank[j] and Y_rank[i] < Y_rank[j]) or (X_rank[i] > X_rank[j] and Y_rank[i] > Y_rank[j]):
                concordant += 1
            elif (X_rank[i] < X_rank[j] and Y_rank[i] > Y_rank[j]) or (X_rank[i] > X_rank[j] and Y_rank[i] < Y_rank[j]):
                discordant += 1

    tau = (concordant - discordant) / np.sqrt((concordant + discordant) * (n * (n - 1) / 2 - discordant))
    return tau



def mutual_information_st(X, Y):
    X = pd.Series(X)
    Y = pd.Series(Y)
    # Compute joint frequency table
    joint_freq = pd.crosstab(X, Y, normalize = True)

    # Compute marginal frequency tables
    p_x = joint_freq.sum(axis = 1)
    p_y = joint_freq.sum(axis = 0)

    # Compute mutual information
    mi = 0.0
    for x in joint_freq.index:
        for y in joint_freq.columns:
            p_xy = joint_freq.loc[x, y]
            if p_xy > 0:  # Avoid division by zero
                mi += p_xy * np.log(p_xy / (p_x[x] * p_y[y]))

    return mi

def statsmodels_corrs (x, y, binary_array):
    df = pd.DataFrame({'x': x, 'y': y})
    corr_matrix = sm.tools.tools.add_constant(df.corr())


    return {
      'Pearson': corr_matrix.loc['x', 'y'],
      'Mutual Information': mutual_information_st(x, y),

    }

Pingouin Library:

In [ ]:
def pingouin_corrs(x, y):
    z = np.random.rand(len(x))
    return {
        'Pearson': pg.corr(x = x, y = y, method ='pearson')['r'].values[0],
        'Spearman': pg.corr(x = x, y = y, method ='spearman')['r'].values[0],
        'Kendall’s Tau': pg.corr( x = x, y = y, method ='kendall')['r'].values[0],
        'Bicor': pg.corr(x = x, y = y, method ='bicor')['r'].values[0],
        'Percbend': pg.corr(x = x, y = y, method ='percbend')['r'].values[0],
        'Shepherd': pg.corr(x = x, y = y, method ='shepherd')['r'].values[0],
        'partial_corr': pg.partial_corr(data = pd.DataFrame({'x': x, 'y': y, 'z': z}), x = 'x', y = 'y', covar = 'z')['r'].values[0],
    }


Dcor Library:

In [ ]:
def spearman_correlation_dcor(x, y):
    """
    Compute Spearman's rank correlation coefficient using dcor for ranking.

    Parameters:
    x (list or np.array or pd.Series): Continuous or ordinal variable x.
    y (list or np.array or pd.Series): Continuous or ordinal variable y.

    Returns:
    float: Spearman's rank correlation coefficient.
    """
    # Convert to pandas Series if they are numpy arrays or lists
    x = pd.Series(x)
    y = pd.Series(y)

    x_rank = x.rank()
    y_rank = y.rank()
    dcor_corr = dcor.distance_correlation(x_rank, y_rank)
    spearman_corr = np.corrcoef(x_rank, y_rank)[0, 1]

    return spearman_corr


def dcor_corrs(x, y):
  return {
      'Spearman': spearman_correlation_dcor(x, y),
      'Distance Correlation' : dcor.distance_correlation(x, y),
    }

Vaex Library:

In [ ]:
def vaex_corrs(x, y):
  dff = vaex.from_arrays(x = x, y = y)
  return {
      'Pearson': dff.correlation(dff.x, dff.y)
  }

XArray Library:

In [ ]:
def xarray_corrs(x, y):
  #x_da = xr.DataArray(x)
  #y_da = xr.DataArray(y)
  x_da = xr.DataArray(x, dims="index")
  y_da = xr.DataArray(y, dims="index")

  # Rank the data
  x_ranked = xr.apply_ufunc(rankdata, x_da)
  y_ranked = xr.apply_ufunc(rankdata, y_da)

  # Calculate Pearson correlation on ranked data
  spearman_corr = xr.corr(x_ranked, y_ranked, dim="index")



  return {
      'Pearson': xr.corr(x_da, y_da).values,
      'Spearman': xr.corr(x_ranked, y_ranked, dim="index").values,

  }


Spark Library:

In [ ]:
def spark_corrs(x, y):
  spark = SparkSession.builder.appName("PearsonCorrelation").getOrCreate()
  data = [(float(x[i]), float(y[i])) for i in range(len(x))]
  df = spark.createDataFrame(data, ["x", "y"])
  return {
      'Pearson': df.select(corr("x", "y")).collect()[0][0],
  }


Geopandas Library:

In [ ]:
def geopandas_corrs(x, y):
  df = pd.DataFrame({'x': x, 'y': y})
  gdf = gpd.GeoDataFrame(df)
  return {
      'Pearson': gdf['x'].corr(gdf['y'], method ='pearson'),
  }


Causal Impact Library:

In [ ]:
def causalimpact_corrs (x, y) :
  df = pd.DataFrame({'x': x, 'y': y})
  return {
      'Pearson': df.corr().loc['x', 'y'],
  }

Pytorch Library:

In [ ]:
def pytorch_corrs(x, y):
    x_tensor = torch.tensor(x)
    y_tensor = torch.tensor(y)
    cov_xy = ((x_tensor - x_tensor.mean()) * (y_tensor - y_tensor.mean())).mean()
    std_x = x_tensor.std()
    std_y = y_tensor.std()

    return {
        'Pearson': (cov_xy / (std_x * std_y)).item(),
    }

Dask Array Library:

In [ ]:
# implements a subset of the NumPy ndarray interface using blocked algorithms, cutting up the large array into many small arrays.
def dask_coors(x, y):
    x_dask = da.from_array(x, chunks = 50)
    y_dask = da.from_array(y, chunks = 50)
    return {
      'Pearson': (((x_dask - x_dask.mean().compute()) * (y_dask - y_dask.mean().compute())).mean().compute()/ (x_dask.std().compute()*y_dask.std().compute())),
    }

Keras and Tensorflow

In [ ]:
def keras_corrs (x, y):
    x_tensor = tf.convert_to_tensor(x, dtype=tf.float32)
    y_tensor = tf.convert_to_tensor(y, dtype=tf.float32)

    # Compute Pearson's correlation coefficient
    mean_x = tf.reduce_mean(x_tensor)
    mean_y = tf.reduce_mean(y_tensor)
    std_x = tf.math.reduce_std(x_tensor)
    std_y = tf.math.reduce_std(y_tensor)

    # Compute the covariance
    covariance = tf.reduce_mean((x_tensor - mean_x) * (y_tensor - mean_y))

    # Pearson correlation coefficient
    pearson_corr = covariance / (std_x * std_y)

    return {
      'Pearson':  (pearson_corr).numpy(),
    }


In [ ]:
results = {
    'Scipy': scipy_corrs(x, y, binary_array),
    'Sklearn': sklearn_corrs(x, y),
    'Pandas': pandas_corrs(x, y, binary_array),
    'Numpy': numpy_corrs(x, y),
    'Statmodels': statsmodels_corrs(x, y, binary_array),
    'Pingouin':pingouin_corrs(x, y),
    'Dcor': dcor_corrs(x, y),
    'Vaex': vaex_corrs(x, y),
    'Xarray': xarray_corrs(x, y),
    'Spark': spark_corrs(x, y),
    'Geopandas': geopandas_corrs(x, y),
    'CausalImpact': causalimpact_corrs(x, y),
    'Pytorch': pytorch_corrs(x, y),
    'Dask': dask_coors(x, y),
    'Keras': keras_corrs(x, y)

}

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/cluster/_supervised.py:66: UserWarning: Clustering metrics expects discrete values but received continuous values for label, and continuous values for target
  warnings.warn(msg, UserWarning)


In [ ]:
results_df = pd.DataFrame(results).T

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


results_df.index.name = 'Library'
results_df.reset_index(inplace = True)
results_df = results_df.rename_axis(None, axis = 1)
pd.set_option('display.float_format', '{:.20f}'.format)

print(results_df)